In [1]:
from scipy import io
import pandas as pd
import os
from tqdm import tqdm
import numpy as np

## Base case multiplet/singlet calculation

In [18]:
# file="../data/Fatemap/stepFourStarcodeShavedReads50.txt"
file="C:/Workspace/Yogesh Rotation/data/fatemapID/stepFourStarcodeShavedReads50.txt"
# TODO: play around [2,4]
umi_cutoff_ratio=3/4e5
umi_diff_threshold=50
dominant_threshold=10
def check_all_unique(target):
    return len(list(set(target)))==len(target)

In [22]:
umi_cutoff_ratio*381348

2.86011

In [13]:
df=pd.read_csv(file,sep='\t')
# df=df[(df['sampleNum']==1)| (df['sampleNum']==2)]
df_sum=df['sampleNum'].value_counts()
all_samples=df['sampleNum'].unique()
# print(all_samples)
df_sum

1    442323
3    381348
2    377422
4    343348
Name: sampleNum, dtype: int64

In [36]:
good_data_ls=[]
for cur_sample_num in df["sampleNum"].unique():
    cur_sample_df=df[df["sampleNum"]==cur_sample_num]
    cur_umi_adjusted_cutoff=round(umi_cutoff_ratio*cur_sample_df.shape[0])
    cur_freq_df=cur_sample_df.groupby(['cellID','BC50StarcodeD8','sampleNum']).size().reset_index()
    cur_freq_df.columns=['cellID',"fatemapID",'sampleNum',"nUMI"]
    cur_good_data=cur_freq_df[cur_freq_df['nUMI']>=cur_umi_adjusted_cutoff].reset_index(drop=True)
    good_data_ls.append(cur_good_data)
good_data=pd.concat(good_data_ls)
good_data=good_data.sort_values(by=["sampleNum", 'cellID', 'fatemapID', 'nUMI']).reset_index(drop=True)
print(good_data.shape)
good_data.head()

(39207, 4)


,cellID,fatemapID,sampleNum,nUMI
0,AAACCCAAGACATAGT,ATTCTAGTTGTAGTACTAGATGATCATCATGTTGTTCTTGGTCTTG...,1,59
1,AAACCCAAGCCGTTAT,ATTCCAGCACGACTACGTGCTGCACTTGGACCTGGTGGAGATCTAC...,1,14
2,AAACCCAAGCCGTTAT,ATTGGACCAGTTGATCAAGTACGTCATGTTCTTGGACGACGTCGTC...,1,8
3,AAACCCAAGTCCGCGT,ATTGTACAACAAGTTCTACTTCCTGTAGTTGAAGGTGTTGCACCAG...,1,13
4,AAACCCACACGCGTGT,ATTCTAGTTGTAGTACTAGATGATCATCATGTTGTTCTTGGTCTTG...,1,16


In [37]:
freq_df=df.groupby(['cellID','BC50StarcodeD8','sampleNum']).size().reset_index()
freq_df.columns=['cellID',"fatemapID",'sampleNum',"nUMI"]
good_data=freq_df[freq_df['nUMI']>=3].reset_index(drop=True)
# good_data.to_csv("../good_data.tsv",sep='\t',index=False)
good_data=good_data.sort_values(by=["sampleNum", 'cellID', 'fatemapID', 'nUMI']).reset_index(drop=True)
print(good_data.shape)
good_data.head()

(39207, 4)


,cellID,fatemapID,sampleNum,nUMI
0,AAACCCAAGACATAGT,ATTCTAGTTGTAGTACTAGATGATCATCATGTTGTTCTTGGTCTTG...,1,59
1,AAACCCAAGCCGTTAT,ATTCCAGCACGACTACGTGCTGCACTTGGACCTGGTGGAGATCTAC...,1,14
2,AAACCCAAGCCGTTAT,ATTGGACCAGTTGATCAAGTACGTCATGTTCTTGGACGACGTCGTC...,1,8
3,AAACCCAAGTCCGCGT,ATTGTACAACAAGTTCTACTTCCTGTAGTTGAAGGTGTTGCACCAG...,1,13
4,AAACCCACACGCGTGT,ATTCTAGTTGTAGTACTAGATGATCATCATGTTGTTCTTGGTCTTG...,1,16


In [28]:
good_data.shape

(39207, 4)

In [8]:
print(good_data.shape)
good_data.head()

(39207, 4)


,cellID,fatemapID,sampleNum,nUMI
0,AAACCCAAGACATAGT,ATTCTAGTTGTAGTACTAGATGATCATCATGTTGTTCTTGGTCTTG...,1,59
1,AAACCCAAGAGAGGTA,ATTCTTCTTGGTGCTCGTGCTGTAGCACCTCGTCTTGCTGTTGGTG...,3,38
2,AAACCCAAGATGCAGC,ATTGGACCTCTAGTTCCACTACGAGTTCATGTAGTTGAAGCACGTG...,2,16
3,AAACCCAAGCCGTTAT,ATTCCAGCACGACTACGTGCTGCACTTGGACCTGGTGGAGATCTAC...,1,14
4,AAACCCAAGCCGTTAT,ATTGGACCAGTTGATCAAGTACGTCATGTTCTTGGACGACGTCGTC...,1,8


In [9]:
## loop through dataframe and examine barcodes freq for cells
cellID2fatemap_dict={}
cellID2fatemap_count_dict={}

# initialize the count dictionaries
for i in all_samples:
    cellID2fatemap_dict[i]={}
    cellID2fatemap_count_dict[i]={}

# loop through each row
for index, row in tqdm(good_data.iterrows(),total=good_data.shape[0]):
    cur_cellID=row['cellID']
    cur_fateID=row['fatemapID']
    cur_sample_num=int(row['sampleNum'])

    # only look at samples 1 and 2 for now
    # if(cur_sample_num not in all_samples):
    #     continue
    if cur_cellID not in cellID2fatemap_dict[cur_sample_num].keys():
        cellID2fatemap_dict[cur_sample_num][cur_cellID]=[cur_fateID]
        cellID2fatemap_count_dict[cur_sample_num][cur_cellID]=1
    else:
        # only add if the fateID is not present in the dict
        if cur_fateID not in cellID2fatemap_dict[cur_sample_num][cur_cellID]:
            cellID2fatemap_dict[cur_sample_num][cur_cellID].append(cur_fateID)
            cellID2fatemap_count_dict[cur_sample_num][cur_cellID]+=1

100%|██████████| 39207/39207 [00:06<00:00, 6042.93it/s]


In [10]:
# sample1_barcodes=list(good_data[good_data["sampleNum"]==1]["cellID"].values)
# sample2_barcodes=list(good_data[good_data["sampleNum"]==2]["cellID"].values)
# multilane_barcodes=set(sample1_barcodes).intersection(sample2_barcodes)
multilane_barcodes=[]
for cur_barcode in good_data["cellID"].unique():
    cur_df=good_data[good_data["cellID"]==cur_barcode]
    if(len(cur_df["sampleNum"].unique())>1):
        multilane_barcodes.append(cur_barcode)

singlets=[]
multiplets=[]
print("Total cells: {}".format(len(good_data["cellID"])))
for sample_id, cur_dict in cellID2fatemap_count_dict.items():
    cur_sample_count=0
    for barcode, count in cur_dict.items():
        # if the barcode is present in more than one lane, remove it
        if barcode in multilane_barcodes:
            continue
        if count==1:
            cur_sample_count+=1
            singlets.append(barcode)
        else:
            multiplets.append(barcode)
    print("Sample {} singlet: {}".format(sample_id,cur_sample_count))
print("Total Singlets: {}\nTotal Multiplets: {}".format(len(singlets),len(multiplets)))

Total cells: 39207
Sample 1 singlet: 4564
Sample 2 singlet: 4366
Sample 3 singlet: 5441
Sample 4 singlet: 2999
Total Singlets: 17370
Total Multiplets: 8200


In [11]:
multilane_barcodes

['AAGAACACAGCTTTGA',
 'AAGAACACAGGCATGA',
 'AAGGAATGTGACACAG',
 'ACGATGTTCTGCGTCT',
 'ACGGTTAGTCGACGCT',
 'AGATCGTAGCTCCGAC',
 'AGGCCACTCCGAACGC',
 'ATTATCCTCGGTAGAG',
 'ATTCCTATCCAATCCC',
 'CAACCTCCAAATCAGA',
 'CAACCTCCACCAGCGT',
 'CAACCTCTCCGTAGGC',
 'CAAGACTGTGTTCATG',
 'CAAGAGGGTCTATGAC',
 'CACAACATCCTCATAT',
 'CACTGAATCGGCACTG',
 'CAGCAATAGGCTCTAT',
 'CCAATGATCTGGGATT',
 'CCGGTAGAGTTACTCG',
 'CGAGTGCCACTCACTC',
 'CTATAGGGTTGGTAGG',
 'CTCTCGAGTGGACCTC',
 'CTTTCAACATGACTAC',
 'GAAACCTAGGTAACTA',
 'GACCAATAGTGGCAGT',
 'GAGCTGCCAAGAGTGC',
 'GAGCTGCCATAACAGA',
 'GATCATGTCGAGCTGC',
 'GATGACTAGCATCCTA',
 'GATGAGGAGTAATTGG',
 'GCAGCTGTCAGAGCGA',
 'GGAATCTGTCCCTCAT',
 'GGGAAGTAGCGAATGC',
 'GTCAAACAGAGTGACC',
 'GTCTACCAGGAGTCTG',
 'GTGCACGCAGAGGCTA',
 'TAAGCCAGTATGCAAA',
 'TACACCCTCACACCGG',
 'TACCCGTAGCCTCTGG',
 'TACTTACTCTTCACGC',
 'TAGATCGGTTAATCGC',
 'TAGGAGGCAACGATTC',
 'TATTTCGTCGGCCCAA',
 'TCACACCGTGCGTCGT',
 'TCACGGGGTGGTAACG',
 'TCATCATCAGAGTCAG',
 'TCATTACCACGTATAC',
 'TCGATTTCATC

In [12]:
with open("./output/step1_singlets.txt","w+") as fp:
    fp.writelines("%s\n"%l for l in singlets)
with open("./output/step1_multiplets.txt","w+") as fp:
    fp.writelines("%s\n"%l for l in multiplets)

## Salvage Scenarios
- Sample combination of fatemapID in the same sample reoccur multiple times
- Shared fatemapID across different samples
- Barcode has one very high UMI count and a few very low UMI count

In [12]:
fatemapID_dict={}
fatemapID_count_dict={}

for cur_multiplet in multiplets:
    cur_df=good_data[good_data["cellID"]==cur_multiplet]
    # only consider when more than 1 fate barcode appears
    n_unique_fatemap_barcodes=len(cur_df["fatemapID"].unique())
    if(n_unique_fatemap_barcodes<2):
        continue
    # create unique key for each fatemap barcode ID
    fatemapID_combo="_".join(sorted(cur_df["fatemapID"].unique()))
    if fatemapID_combo not in fatemapID_dict:
        fatemapID_dict[fatemapID_combo]=[cur_multiplet]
        fatemapID_count_dict[fatemapID_combo]=1
    else:
        fatemapID_dict[fatemapID_combo].append(cur_multiplet)
        fatemapID_count_dict[fatemapID_combo]+=1


In [13]:
two_barcode_singlets_count=0
two_barcode_singlets=[]
for key, value in fatemapID_count_dict.items():
    if value>=2:
        two_barcode_singlets_count+=value
        two_barcode_singlets.extend(fatemapID_dict[key])
        if(fatemapID_dict[key] in two_barcode_singlets):
            print(fatemapID_dict[key])
print(check_all_unique(two_barcode_singlets))
print("UMI cutoff: {}".format(umi_cutoff))
print("Count of salvaged cells: {}\nProportion of salvaged cells: {}".format(two_barcode_singlets_count,
             two_barcode_singlets_count/len(good_data["cellID"].unique())))

True
UMI cutoff: 3
Count of salvaged cells: 2678
Proportion of salvaged cells: 0.10449100628194623


In [14]:
two_barcode_singlets

['AAACCCAAGCCGTTAT',
 'ACATCGACATTCAGGT',
 'AAAGAACAGGTTGCCC',
 'AACAAAGCATTCCTAT',
 'AATCGACCACAATTCG',
 'ACCTACCTCTTCCTAA',
 'ACTTTGTCAAATGATG',
 'AGGGCTCTCTGTCCCA',
 'ATGCCTCTCTACCAGA',
 'CAGCCAGGTCTAATCG',
 'CAGGCCACAACACGAG',
 'CATCGCTGTACTGACT',
 'CCAAGCGCAGCTCCTT',
 'CCTAAGACAGACGATG',
 'CGAGGCTTCCGTAGTA',
 'CGAGTTATCAGTGATC',
 'CGTAGTAAGACCTTTG',
 'CTACGGGGTATAGGAT',
 'CTCAGGGAGTAGACCG',
 'CTCCTTTGTCGTGGAA',
 'GAACGTTAGGACGCAT',
 'GAAGAATTCTCGACCT',
 'GACAGCCGTCCAACGC',
 'GACCAATAGTCAGCGA',
 'GATTGGTGTCCTTAAG',
 'GATTGGTGTTGGACCC',
 'GCTTGGGCACACCGAC',
 'GCTTGGGCAGGGCTTC',
 'GGAGGTACAAAGCAAT',
 'GGGACTCCAGTGTGCC',
 'GGGTCACTCCATGAGT',
 'GGGTGTCCAAGTGATA',
 'GTAGGAGCATACCATG',
 'GTCATCCAGATTGTGA',
 'GTCATTTGTCTACGAT',
 'GTTACCCTCCCTCATG',
 'TAGACCAGTGCCTTCT',
 'TATACCTAGGTTCCAT',
 'TCATATCTCCTATTTG',
 'TCATTGTTCATTGCCC',
 'TCATTTGCAGCGATTT',
 'TCCACGTAGCAGCACA',
 'TCCACGTAGGTATCTC',
 'TCCTGCAAGCCTATCA',
 'TCGAACATCAACCGAT',
 'TCTGCCAGTATGAAGT',
 'TGAACGTAGGTACAGC',
 'TGAATGCTCGA

In [105]:
singlets_step2=list(set(singlets).union(set(two_barcode_singlets)))
multiplets_step2=list(set(multiplets).difference(set(two_barcode_singlets)))

In [106]:
print("Total Singlets: {}\nTotal Multiplets: {}".format(len(singlets_step2),len(multiplets_step2)))

Total Singlets: 20048
Total Multiplets: 5522


In [107]:
UMI_thres_singlets=[]
for cur_multiplet in multiplets_step2:
    cur_df=good_data[good_data["cellID"]==cur_multiplet]
    cur_UMI_counts=cur_df['nUMI'].values
    cur_median_UMI=np.median(cur_UMI_counts)
    if((np.max(cur_UMI_counts)-cur_median_UMI)>umi_diff_threshold):
        # check if there is more than 1 dominant
        dominant_count=0
        for cur_UMI in cur_UMI_counts:
            cur_diff=cur_UMI-cur_median_UMI
            if(cur_diff>umi_diff_threshold or cur_UMI>dominant_threshold):
                dominant_count+=1

        if(dominant_count==1):
            UMI_thres_singlets.append(cur_multiplet)


In [108]:
singlets_step3=list(set(singlets_step2).union(set(UMI_thres_singlets)))
multiplets_step3=list(set(multiplets_step2).difference(set(UMI_thres_singlets)))
print("Total Singlets: {}\nTotal Multiplets: {}".format(len(singlets_step3),len(multiplets_step3)))

Total Singlets: 20477
Total Multiplets: 5093


In [260]:
with open("./output/step2_singlets.txt","w+") as fp:
    fp.writelines("%s\n"%l for l in singlets_step2)
with open("./output/step2_multiplets.txt","w+") as fp:
    fp.writelines("%s\n"%l for l in multiplets_step2)

## A few sanity checks

In [16]:
t="AAACCCAAGCCGTTAT"
t_df=good_data[good_data["cellID"]==t]
t_id=fatemapID_combo="_".join(sorted(t_df["fatemapID"].unique()))
print(t_df["fatemapID"].unique())
t_df

['ATTCCAGCACGACTACGTGCTGCACTTGGACCTGGTGGAGATCTACCTGG'
 'ATTGGACCAGTTGATCAAGTACGTCATGTTCTTGGACGACGTCGTCCTCA']


,cellID,fatemapID,sampleNum,nUMI
3,AAACCCAAGCCGTTAT,ATTCCAGCACGACTACGTGCTGCACTTGGACCTGGTGGAGATCTAC...,1,14
4,AAACCCAAGCCGTTAT,ATTGGACCAGTTGATCAAGTACGTCATGTTCTTGGACGACGTCGTC...,1,8


In [17]:
fatemapID_dict[t_id]

['AAACCCAAGCCGTTAT', 'ACATCGACATTCAGGT']

In [18]:
t="ACATCGACATTCAGGT"
t_df=good_data[good_data["cellID"]==t]
t_id=fatemapID_combo="_".join(sorted(t_df["fatemapID"].unique()))
print(t_df["fatemapID"].unique())
t_df

['ATTCCAGCACGACTACGTGCTGCACTTGGACCTGGTGGAGATCTACCTGG'
 'ATTGGACCAGTTGATCAAGTACGTCATGTTCTTGGACGACGTCGTCCTCA']


,cellID,fatemapID,sampleNum,nUMI
2335,ACATCGACATTCAGGT,ATTCCAGCACGACTACGTGCTGCACTTGGACCTGGTGGAGATCTAC...,1,43
2336,ACATCGACATTCAGGT,ATTGGACCAGTTGATCAAGTACGTCATGTTCTTGGACGACGTCGTC...,1,43
